<a href="https://colab.research.google.com/github/krithiyer/safe-elections/blob/main/cs294s_2A_SimilarityCalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Extract word vector and calculate similarity

Extract question embedding from different layers of BERT model finetuned on Quora Question Pairs dataset.

Use different algorithmes to evaluate text similarity - match user question with questions in our (election / covid) curated FAQ

Connect to google drive and load the saved model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/cs294s/

/content/drive/My Drive/cs294s


In [ ]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


install and import all needed packages

In [ ]:
pip install transformers

     |████████████████████████████████| 1.1MB 8.4MB/s 
     |████████████████████████████████| 890kB 33.1MB/s 
     |████████████████████████████████| 1.1MB 55.3MB/s 
     |████████████████████████████████| 3.0MB 50.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=63fcd01e012e28657b63b7280a5c0eeb71515d371451a5af621949e45b4cc94e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

The model saved at './models_saved_qqp_oct9Final/' was fine tuned on QQP dataset for 4 epoch. 

In [ ]:


# The name of the folder containing the model files.
input_dir = './models_saved_qqp_oct9Final/'

# Load our fine-tuned model, and configure it to return the "hidden states", 
# from which we will be taking our text embeddings.
# num_labels = 2,  is this needed while loading saved models
model = BertForSequenceClassification.from_pretrained(
    input_dir,
    output_hidden_states = True, # Whether the model returns all hidden-states.
) 

# Load the tokenizer. we have not saved it.
#tokenizer = BertTokenizer.from_pretrained(output_dir)

# model.cuda()

# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

Question to Embedding function. 

will take a question (from a user or from the curated FAQ list) and compute the word embedding vector for the question. The embedding vector of the user-question will be matched against the embeddings vectors of the questions from the FAQ collection to select responses.

The embedding from the [CLS] token from the top layer is used. (Other experiments with averaging embeddings from different layers will be considered later)


In [ ]:
import torch
from keras.preprocessing.sequence import pad_sequences

def question_to_embedding(tokenizer, model, in_text):
    '''
    Uses the provided BERT `model` and `tokenizer` to generate a vector 
    representation of the input string, `in_text`.

    Returns the vector stored as a numpy ndarray.
    '''

    MAX_LEN = 256

    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Truncate the sentence to MAX_LEN if necessary.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end. (After truncating!)
    #   (4) Map tokens to their IDs.
    input_ids = tokenizer.encode(
                        in_text,                    # Sentence to encode.
                        add_special_tokens = True,  # Add '[CLS]' and '[SEP]'
                        max_length = MAX_LEN,       # Truncate all sentences. 
                        truncation = True                       
                   )    

    # Pad our input tokens. Truncation was handled above by the `encode`
    # function, which also makes sure that the `[SEP]` token is placed at the
    # end *after* truncating.
    # Note: `pad_sequences` expects a list of lists, but we only have one
    # piece of text, so we surround `input_ids` with an extra set of brackets.
    results = pad_sequences([input_ids], maxlen=MAX_LEN, dtype="long", 
                              truncating="post", padding="post")
    
    # Remove the outer list.
    input_ids = results[0]

    # Create attention masks    
    attn_mask = [int(i>0) for i in input_ids]
    
    # Cast to tensors.
    input_ids = torch.tensor(input_ids)
    attn_mask = torch.tensor(attn_mask)

    # Add an extra dimension for the "batch" (even though there is only one 
    # input in this batch.)
    input_ids = input_ids.unsqueeze(0)
    attn_mask = attn_mask.unsqueeze(0)


    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Copy the inputs to the GPU

    input_ids = input_ids.to(device)
    attn_mask = attn_mask.to(device)
    
    # no need to build the backwards graph. 
    # this will make the model run a bit faster.

    with torch.no_grad():        

        # Forward pass, return hidden states and predictions.
        # Note - since we are not supplying the lables to the model
        # it will not return any loss.

        logits, encoded_layers = model(
                                    input_ids = input_ids, 
                                    token_type_ids = None, 
                                    attention_mask = attn_mask)
        
    # Retrieve our sentence embedding--
    # experiment 1 : take the `[CLS]` embedding from the final
    # layer.
    # we can set the parameters below for different experimentation
    # for embeddings.
    
    layer_i = 12 # The last BERT layer before the classifier.
    batch_i = 0 # Only one input in the batch.
    token_i = 0 # The first token, corresponding to [CLS]
        
    # Grab the embedding.
    vec = encoded_layers[layer_i][batch_i][token_i]

    # Move to the CPU and convert to numpy ndarray.
    vec = vec.detach().cpu().numpy()

    return(vec)

we also need to generate word embedding vectors for each question in the FAQ database.

we need to read in the FAQ database - hopefully the FAQ database will be pre-processed in a notebook and readily aviable for reading

To Do: read the FAQ database from a saved file

In [ ]:
pwd

'/content/drive/My Drive/cs294s'

Initial curated QA consists of 1648 questions related to Covid Safety, Election Deadlines, etc.

Questions scraped from the Secretary of State web sites (election data)
and Covid safety data from different sites.

Some Covid related questions taken from  https://openreview.net/pdf?id=qd51R0JNLl   (not all questions used - some dont have answers)


In [ ]:
import pandas
election_faq_df = pandas.read_excel('./master_test1_v1_QA.xlsx')
election_faq_df.head()

,Question,Answer
0,"Due to COVID-19, can an election official requ...","No, an election official cannot require a vote..."
1,Can election officials require voters to wear ...,"No, you cannot require voters to wear face cov..."
2,May election officials require an assistant/in...,"No, just as you cannot require a voter to use ..."
3,If an election worker cannot identify a voter ...,The election judge has discretion to ask the v...
4,How should election officials sanitize equipme...,The SOS recommends that election officials san...


In [ ]:
proc_data_df = election_faq_df.dropna()

In [ ]:
election_faq_df.shape

(1647, 2)

In [ ]:
faq_q_embeddings = []
for row in proc_data_df.itertuples():
  vec = question_to_embedding(tokenizer, model, row.Question)
  faq_q_embeddings.append(vec)

In [ ]:
len(faq_q_embeddings)

1646

In [ ]:
import numpy as np

# Convert the list of vectors into a 2D array.
vecs = np.stack(faq_q_embeddings)

vecs.shape

(1646, 768)

In [ ]:
np.save('./faq_q_oct12_embeddings.npy', vecs)

Need to read FAQ questions from the database - and iterate one question per call

Similar Question Search

Cosine similarity is one option.

Now that we have our comments all vectorized, we are ready to make them "searchable".

We do this using a technique called "k-Nearest Neighbor Search" or "k-NN". Simply put, we use a distance metric such as Euclidean distance, calculate that distance between our "query" vector and all of the vectors to be searched, then sort the distances to find the closest matches.

All of those distance calculations can make k-NN search computationally expensive and slow. There are a number of libraries out there for accelerating k-NN using carefully optimized code and / or approximation techniques.

I personally like the FAISS (Facebook AI Similarity Search) library, in part because it has a really excellent GPU implementation.

Using the GPU, we can perform "brute-force" k-NN search (meaning no approximation techniques which compromise on accuracy) on this dataset quickly.

more info on faiss

https://github.com/facebookresearch/faiss/wiki/Getting-started



In [ ]:
!pip install faiss

     |████████████████████████████████| 4.7MB 9.8MB/s 


In [ ]:
!pip install faiss-gpu

     |████████████████████████████████| 35.5MB 87kB/s 


In [ ]:
import faiss
cpu_index = faiss.IndexFlatL2(vecs.shape[1])

In [ ]:
cpu_index.add(vecs)

Now that we have built the index for FAQ questions, let us experiment with a few questions

following very trival example, using vector 8 to query.

In [ ]:
D, I = cpu_index.search(vecs[8].reshape(1, 768), k=5) 

In [ ]:
for i in range(I.shape[1]):

    # Look up the question row number for this result.
    result_i = I[0, i]
    print (result_i)
    print(proc_data_df['Question'][result_i])
    print(proc_data_df['Answer'][result_i])

8
May a poll watcher observe a voter being assisted in preparing their ballot?
A watcher may not be present at the voting station when a voter is preparing the voter’s ballot or is being assisted by a person of the voter’s choice, including by a person also serving as an interpreter at the voting station. 
4
How should election officials sanitize equipment after use?
The SOS recommends that election officials sanitize equipment after each use, particularly if a voter is showing any signs or symptoms of COVID-19. Please contact your vendor about the specific procedures you should follow to clean and sanitize the equipment being used.
7
May election officials place social distancing requirements on a poll watcher?
No, you cannot place social distancing requirements on a poll watcher. A person commits an offense if the person serves in an official capacity at a location at which the presence of watchers is authorized and knowingly prevents a watcher from observing an activity the watcher 

In [ ]:
user_question = 'How can I avoid getting Covid  '

user_query_vec = question_to_embedding(tokenizer, model, user_question)

D, I = cpu_index.search(user_query_vec.reshape(1, 768), k=5) 

for i in range(I.shape[1]):

    # Look up the comment row number for this result.
    result_i = I[0, i]
    print (result_i)
    print(proc_data_df['Question'][result_i])
    print(proc_data_df['Answer'][result_i])


779
what will be the educational impacts of covid
read or watch local media sources that report school dismissals or and watch for communication from your childs school.
865
how do i stop covid
the best way to prevent illness is to avoid being exposed to the virus. , stay at least 6 feet away from others, and wash your hands often. , regularly and thoroughly clean your hands with an alcohol based hand rub or wash them with soap and water. maintain at least 3 feet distance between yourself and anyone who is coughing or sneezing.
748
how can people help stop stigma related to covid
people can fight stigma by providing social support in situations where you notice this is occurring.
208
can a person test negative and later test positive for covid
a negative result means that the virus that causes covid was not found in the persons sample. in the early stages of infection, it is possible the virus will not be detected. 
835
what should i do if i had close contact with someone who has covid

In [ ]:
user_question = 'Can i safely vote during covid  '

user_query_vec = question_to_embedding(tokenizer, model, user_question)

D, I = cpu_index.search(user_query_vec.reshape(1, 768), k=5) 

for i in range(I.shape[1]):

    # Look up the comment row number for this result.
    result_i = I[0, i]
    print (result_i)
    print(proc_data_df['Question'][result_i])
    print(proc_data_df['Answer'][result_i])

902
can the covid virus spread through drinking water
there is no evidence that covid can be spread to humans through the use of pools and hot tubs.
448
how long is the incubation period for covid
distribution is gamma distributed with mean 5.1 days and coefficient of variation, between 2 and 14 days, most estimates of the incubation period for covid range from 1 14 days, most commonly around five days.
167
are blood tests useful to diagnose covid
 there are many tests being used to diagnose covid that the u.s. food & drug administration (fda) has authorized for use during the current emergency. all of these diagnostic tests identify the virus in samples from the respiratory system, such as from nasal or nasopharyngeal swabs, all of these diagnostic tests identify the virus in samples from the respiratory system, such as from nasal or nasopharyngeal swabs. , rrt pcr test,  reverse transcription polymerase chain reaction, the 2109 ncov rna was readily detected in the blood (6 of 57 pati

In [ ]:
user_question = 'Can I request mailin ballot because of covid  '

user_query_vec = question_to_embedding(tokenizer, model, user_question)

D, I = cpu_index.search(user_query_vec.reshape(1, 768), k=5) 

for i in range(I.shape[1]):

    # Look up the comment row number for this result.
    result_i = I[0, i]
    print (result_i)
    print(proc_data_df['Question'][result_i])
    print(proc_data_df['Answer'][result_i])

773
what can i do to prepare my family for a covid outbreak
practice everyday preventive actions to help reduce your risk of getting sick and remind everyone in your home to do the same., create a household plan of action to help protect your health and the health of those you care about in the event of an outbreak of covid in your community .
442
how can i protect my child from covid infection
discourage children and teens from gathering in other public places while school is dismissed to help slow the spread of covid in the community. encourage frequent handwashing and follow other prevention tips., discourage children and teens from gathering in other public places while school is dismissed to help slow the spread of covid in the community., you can encourage your child to help stop the spread of covid by teaching them to do the same things everyone should do to stay healthy.
1589
When can I start voting in-person in Washington
In Washington, in-person voting starts 2020-10-16
788
a